In [39]:
from astropy.io import fits
from astropy.stats import biweight_midvariance as bwmv
from astropy.stats import median_absolute_deviation as mad
import astroalign as aa
import cv2
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from skimage.registration import phase_cross_correlation
# Shift the image
from scipy.ndimage import shift
import pandas as pd
import os

def fitsread(f, header=False):
    with fits.open(f) as hdul:
        data = hdul[0].data
        if header:
            h = hdul[0].header
            return data, h
        else:
            return data
        
def rescale(image, mref, sref, roi=None):
    imageV = image if roi is None else image[roi]
    m1 = np.median(imageV)
#     s1 = np.sqrt(bwmv(imageV))
    s1 = mad(imageV)
    ki = sref/s1
    rimage = ki*(image - m1) + mref
    return rimage

IMAX = 65535

In [2]:
# Masters
cal_dir = Path(os.environ['DATA'], 'DDS', 'Taka', 'Calibration')
dark600f = Path(cal_dir, 'masterDark600_bin1_average_Jan_2019.fits')
HFlatf = Path(cal_dir, 'masterHFlat_bin1_Mar_3_2019.fits')
print(str(cal_dir), cal_dir.exists())
# Load master files - Dark is not Bias-subtracted, but Master flat is bias-subtracted. 
Dark = fitsread(dark600f)
HFlat = fitsread(HFlatf)

C:\Data\DDS\Taka\Calibration True


In [3]:
# Lights
lights_dir = Path(os.environ['DATA'], 'DDS', 'Taka', 'Lights', 'IC_1805')
Hlights_dir = Path(lights_dir, 'H')
print(str(Hlights_dir), Hlights_dir.exists())
Hlightsf = sorted(Hlights_dir.rglob('H*.fit'))
print('number of lights: ', len(Hlightsf))
# Load 1st image to get reference statistics
im0 = fitsread(Hlightsf[0]) - Dark
# image size
sz = im0.shape

fov1 = np.s_[1500:1500+512, 2000:2000+512]
fov2 = np.s_[0:256, 0:256]
fov3 = np.s_[-256:, -256:]

C:\Data\DDS\Taka\Lights\IC_1805\H True
number of lights:  73


In [38]:
Dark_s = fitsread(Path(cal_dir, 'CCD_Souhayl/masterDARK_BIN1_600s_-25deg.fit'))
Dark_s.min(), Dark_s.max()

(0.0009082852, 0.9897744)

In [4]:
d = fitsread(Hlights_dir/'H_IC 1805_600s.06001.fit')

In [5]:
np.median(d)

692.0

### Load selected files upon rejection from above jpeg previews (and optionally Maxselector)

In [22]:
Hlightsf = sorted(Hlights_dir.glob('H*.fit'))
selectf = Path(Hlights_dir, 'selection_raphael.txt')
with open(selectf) as file: lines = [line.rstrip('\n') for line in file]
print('index of rejected file index: ', lines)
print('Nb of selected files: ', len(Hlightsf) - len(lines))
Hf = [f for f in Hlightsf if not any(x in str(f) for x in lines)]
print(len(Hf))
Hf

index of rejected file index:  ['06028', '06029', '06058', '06059', '06060', '06061', '06063', '06064', '06066', '06067', '06068', '06069', '06070', '06072', '06073', '06074', '06076', '06077', '06081', '06082', '06083', '06095', '06096', '06097', '06103', '06105', '06106', '06107', '06108']
Nb of selected files:  43
43


[WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06001.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06002.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06003.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06004.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06005.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06006.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06007.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06008.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06009.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06010.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06011.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06012.fit'),
 WindowsPath('C:/Data/DDS/Taka/Lights/IC_1805/H/H_IC 1805_600s.06013.fit'),
 WindowsPath

In [23]:
# Load lights from curated list
HLights = np.array([fitsread(f) for f in Hf])

In [24]:
HCal1 = HLights - Dark

In [25]:
# Flat Field and bring back to pre-Flat-field equivalent histograms
HCal2 = (HCal1 / HFlat).astype(np.float32)

In [26]:
Hcaldir = Path(Hlights_dir, 'calibrated')
Hcaldir.mkdir(parents=False, exist_ok=True)
# Export calibrated files to FITS files
for i, f in enumerate(Hf):
    fits.writeto(Path(Hcaldir, f'cal.{f.stem}.fits'), HCal2[i].astype(np.float32), overwrite=True)

In [18]:
Hregdir = Path(Hlights_dir, 'registered')
Hregdir.mkdir(parents=False, exist_ok=True)
Path(Hregdir, 'jpegs').mkdir(parents=False, exist_ok=True)

# Rescale the reference image
mref = np.median(HCal1[0])
# sref = np.sqrt(bwmv(HCal1[0]))
sref = mad(HCal1[0])
ref = rescale(HCal1[0], mref, sref)

vmin = np.percentile(HCal2[0], 0.1)
vmax = np.percentile(HCal2[0], 99.8)
for i, f in enumerate(Hf[0:]):
    
    image2 = HCal2[i]
    
    image2 = rescale(image2, mref, sref)
    
    p, (pos_img, pos_img_rot) = aa.find_transform(image2, ref)
    image2 = cv2.warpPerspective(image2, p.params, (sz[1],sz[0]), flags=cv2.INTER_LANCZOS4);
    fits.writeto(Path(Hregdir, f'reg.{f.stem}.fits'), image2.astype(np.float32), overwrite=True)
    
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))
    axs[0].imshow(image2[fov1], vmin=vmin, vmax=vmax, cmap='gray')
    axs[1].imshow(image2[fov2], vmin=vmin, vmax=vmax, cmap='gray')
    axs[2].imshow(image2[fov3], vmin=vmin, vmax=vmax, cmap='gray')
    axs[0].axis('off')
    axs[1].axis('off')
    axs[2].axis('off')
    plt.tight_layout()
    plt.savefig(Path(Hregdir, 'jpegs', f'{f.stem}.reg.aa.jpg'))
    plt.close()

MaxIterError: Max iterations exceeded while trying to find acceptable transformation.

In [27]:
Hregf = sorted(Hregdir.rglob('reg*.fits'))
len(Hregf)

43

In [28]:
Hreg = np.array([fitsread(f) for f in Hregf])

In [29]:
Hstack_median = np.median(Hreg, axis=0)
Hstack_avg = Hreg.mean(axis=0)

In [30]:
Hstack_median = Hstack_median / Hstack_median.max()
Hstack_median[Hstack_median <0]=0
Hstack_avg = Hstack_avg / Hstack_avg.max()
Hstack_avg[Hstack_avg<0] = 0

In [32]:
fits.writeto(Path(Hlights_dir, 'stacked', 'Hstack_median_Raphael_mad_D.fits'), Hstack_median.astype(np.float32), overwrite=True)
fits.writeto(Path(Hlights_dir, 'stacked', 'Hstack_avg_Raphael_mad_D.fits'), Hstack_avg.astype(np.float32), overwrite=True)
H16 = np.uint16(np.round(Hstack_avg*IMAX))
fits.writeto(Path(Hlights_dir, 'stacked', 'Hstack_avg_Raphael_mad_D_uint16.fits'), H16, overwrite=True)

In [33]:
madH = mad(Hreg, axis=0)
mask = np.abs(Hreg - Hstack_median) < 6*madH
mHreg = np.ma.masked_array(Hreg, mask=~mask)
mHstack2 = mHreg.mean(axis=0).filled(0)

mHstack2 = mHstack2 / mHstack2.max()
mHstack2[mHstack2<0] = 0
fits.writeto(Path(Hlights_dir, 'stacked', 'Hstack_mad_avg_rej_6mad_Raphael.fits'), mHstack2.astype(np.float32), overwrite=True)

In [ ]:
# mHstack2_med = np.ma.median(mHreg, axis=0).filled(0)
# mHstack2_med = mHstack2_med / mHstack2_med.max()
# mHstack2_med[mHstack2_med<0] = 0
# fits.writeto(Path(Hlights_dir, 'stacked', 'Hstack_mad_median_rej_mad_Raphael.fits'), mHstack2_med.astype(np.float32), overwrite=True)

In [34]:
pid = fitsread(Path(Hlights_dir, 'stacked', 'H_IC_1805_stacked_PI.fit'))
mpid = np.median(pid)
spid = mad(pid)
Havg = rescale(Hstack_avg, mpid, spid)

In [35]:
pid.min(), pid.max()

(0.0, 1.0)